In [33]:
import pandas as pd
from rapidfuzz import fuzz
from rich.progress import Progress
from rich.console import Console
from dataclasses import dataclass
import os
import re

console = Console()

# Read in the datasets to two separate data frames
df1 = pd.read_csv(
    "APN and OPS Combined Shop List for Loading.csv", encoding="mac_roman"
)
df1.rename(columns={'ÔªøAccount Name': 'Account Name'}, inplace=True)

df2 = pd.read_csv("CRM Shops Normalized Addresses.csv", encoding="mac_roman")


full_dir = "full_data_output"
final_dir = "final_data_output"

if not os.path.exists(full_dir):
    os.makedirs(full_dir)

if not os.path.exists(final_dir):
    os.makedirs(final_dir)


class BaseRange:
    min: int
    max: int = 100
    weight_address: float = 0.6
    weight_name: float = 0.4

    def __post_init__(self):
        if self.min > self.max:
            raise ValueError("min must be less than max")


class Confident(BaseRange):
    name_min = 75
    address_min = 85
    almost_exact_threshold = 97
    weighted_min = 80


class Maybe(BaseRange):
    name_min = 60
    address_min = 70
    almost_exact_threshold = 97
    weighted_min = 70


# Concatenate the columns
df1['Address'] = df1[['Street', 'City', 'State/Province', 'Zip', 'Country']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

#print(df1.Address)

df2['Address'] = df2[['Street', 'City', 'State/Province', 'Address 1: ZIP/Postal Code', 'Country']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

#print(df2.Address)

address_normalize_list=["st.","st", "street", "blvd.", "blvd", "boulevard", "ave.", "ave", "avenue", "road", "rd.", "rd", "lane", "ln", "ln."]
normalization_replacement={"street":"st", "road":"rd", "avenue": "ave", "boulevard": "blvd", "lane":"ln"}

#Normalizing Function for Account and Address Uppercase to Lowercase
def normalize(accountname, address):
    accountname = accountname.lower()
    address= address.lower() 
    normalized_address = "" 
    for word in address.split(" "):
       word=re.sub(r"[^A-Za-z]+", '', word)
       try:
            replacement = normalization_replacement[word]
            normalized_address = address.replace(word, replacement)
       except Exception as e:
           pass
    if not normalized_address:
        normalized_address=address
    return(accountname, normalized_address)

Index(['Account Name', 'Street', 'City', 'State/Province', 'Zip', 'Country',
       'Address'],
      dtype='object')

In [34]:

confident_matches = []
maybe_matches = []

df1_account_names = [x.strip() for x in list(df1["Account Name"]) if isinstance(x, str)]
df1_addresses = [x.strip() for x in list(df1["Address"]) if isinstance(x, str)]
df2_account_names = [x.strip() for x in list(df2["Account Name"]) if isinstance(x, str)]
df2_addresses = [x.strip() for x in list(df2["Address"]) if isinstance(x, str)]

with Progress() as progress:
    task = progress.add_task("[red]Matching...", total=len(df1_account_names))

    for name, address in zip(df1_account_names, df1_addresses):
        for name2, address2 in zip(df2_account_names, df2_addresses):
            row = {"Account Name": name, "Full Address": address}
            full_row = {
                "Account Name": name,
                "Account Name 2": name2,
                "Full Address": address,
                "Full Address 2": address2,
            }
            name, address= normalize(name, address) 
            name2, address2= normalize(name2, address2)
            # Calculate the ratio of similarity between the pairs of names and addresses
            name_ratio, address_ratio = fuzz.ratio(name, name2), fuzz.ratio(
                address, address2
            )

            # Calculate the weighted ratio of similarity
            weighted_ratio = (
                name_ratio * BaseRange.weight_name
                + address_ratio * BaseRange.weight_address
            )

            # Check if either ratio is above the almost exact threshold
            almost_exact_match = (
                name_ratio >= Confident.almost_exact_threshold
                or address_ratio >= Confident.almost_exact_threshold
            )

            if (
                (weighted_ratio >= Confident.weighted_min and almost_exact_match)
                or (name_ratio >= Confident.name_min
                    and address_ratio >= Confident.address_min)
            ):
                confident_matches.append(full_row)
                break
            elif weighted_ratio >= Maybe.weighted_min:
                maybe_matches.append(full_row)
                break

        progress.update(task, advance=1)


pd.DataFrame(confident_matches).to_csv(f"{full_dir}/confident_matches.csv")
pd.DataFrame(maybe_matches).to_csv(f"{full_dir}/maybe_matches.csv")

confident_matches_final = [{k: v for (k, v) in x.items() if (k == "Account Name" or k == "Full Address")} for x in confident_matches]
maybe_matches_final = [{k: v for (k, v) in x.items() if (k == "Account Name" or k == "Full Address")} for x in maybe_matches]

pd.DataFrame(confident_matches_final).to_csv(f"{final_dir}/confident_matches.csv")
pd.DataFrame(maybe_matches_final).to_csv(f"{final_dir}/maybe_matches.csv")


Output()

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored